**SWATHI P**

**CB.EN.U4CSE19353**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
revs = pd.read_csv('/content/drive/MyDrive/Machine Learning/revs_final.csv',  error_bad_lines=False)
revs.head()

,reviewerID,asin,reviews,rating,sentiment
0,A2QK1U70OJ74P,B000FA64PA,well written interesting to see sideous thro...,3,0
1,A3SZMGJMV0G16C,B000FA64PK,troy denning s novella recovery was originally...,3,1
2,A38Z3Q6DTDIH9J,B000FA64PK,another well written ebook by troy denning bu...,3,0
3,A3SZMGJMV0G16C,B000FA64QO,with ylesia a novella originally published in...,2,0
4,A22CW0ZHY3NJH8,B000FA64QO,the events of ylesia take place during dest...,3,0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = revs.loc[revs['sentiment'] == 1]
df.head()

,reviewerID,asin,reviews,rating,sentiment
1,A3SZMGJMV0G16C,B000FA64PK,troy denning s novella recovery was originally...,3,1
5,A3SZMGJMV0G16C,B000FBFMVG,originally published as an e book coinciding w...,3,1
7,A3SZMGJMV0G16C,B000FC1BN8,timothy zahn s fool s bargain is a short story...,3,1
13,ADXAE3UOGPUN4,B000FC26RI,did not impress me that much as i have read t...,3,1
24,A9FVWRHMM550F,B000GFK7L6,i thought the n y times was a morning daily ...,1,1


In [ ]:
counts1=df['reviewerID'].value_counts() 
counts=df['asin'].value_counts()

In [ ]:
counts1

A13QTZ8CIMHHG4    224
A2WZJDFX12QXKD    177
A37LY77Q2YPJVL    150
A320TMDV6KCFU     143
A3PTWPKPXOG8Y5    131
                 ... 
A3RZ0HT9P7VFR0      1
A2BLQP7CWQX8AF      1
AJ9IPJR2KHQNR       1
A3AOJ120MH62T3      1
AY63QKPC0760Y       1
Name: reviewerID, Length: 50534, dtype: int64

In [ ]:
counts

B006GWO5WK    199
B00BTIDW4S    131
B00H0V069M     96
B00BSX4U04     87
B00JDYC5OI     80
             ... 
B00H324XOW      1
B009W2TFS0      1
B00K7BK1GU      1
B00GV3XQ8I      1
B0087NVG2I      1
Name: asin, Length: 48832, dtype: int64

In [ ]:
df1=df[df['reviewerID'].isin(counts1[counts1 >=30].index)]
df1.shape

(12201, 5)

In [ ]:
df2=df1[df1['asin'].isin(counts[counts >= 10].index)]
df2.shape

(1681, 5)

In [ ]:
# cold user - based on popularity
ratings_sum = pd.DataFrame(df2.groupby(['asin'])['rating'].sum()).rename(columns = {'rating': 'ratings_sum'})
top10 = ratings_sum.sort_values('ratings_sum', ascending = False).head(10)
top10


,ratings_sum
asin,
B00BCFWJ7E,28
B00BT0J8ZS,27
B00DXIFORA,25
B00HKMUFU6,25
B00DQTZOH6,24
B005C5YZ86,22
B00E7IWEFU,20
B00CEF014E,20
B00CJ3UPTW,19


In [ ]:
ratingsd=df2.pivot(index='asin',columns= 'reviewerID',values='rating').fillna(0)
ratingsd.head()

reviewerID,A10Y597K071WTQ,A1168ILANAUMV8,A11US7VEC93CT6,A135NSTCX38DEI,A13D5S4OXQUPRM,A13QTZ8CIMHHG4,A141H51I3H4B1S,A14I311IIUMBWC,A14QJKY3DA19AP,A156DL305O94MP,A162RUEBFRKZ17,A16NOA3DB7G1SR,A17126555O7KXC,A18JHVX925K101,A197W8MNF706HT,A19GJX6TTSK0C0,A19UT0OPQCDR8,A1A2UUBPN2AMTO,A1AH7FPROGNXH8,A1ARJBC8VD5P8P,A1BQO66R6OLCCW,A1CGOR398UH1IB,A1CQ8WTACDSN6V,A1CUXURVHSE9QP,A1CXGXJZZ3AXR5,A1D1GP1B7UH5NE,A1E1LEVQ9VQNK,A1E5ZR1Z4OQJG,A1EE05KGTY3VY6,A1ENV91MFAEVA3,A1EZ9Z9UF4EVMG,A1FJ6YD0TL90GN,A1FTB3KM7X78OX,A1H0PTT0NVO9J7,A1I5JKKMD84DRU,A1J4ZZVGTNBQFE,A1JKGTL51HHTU1,A1JLU5H1CCENWX,A1JMSX54DO3LOP,A1JQG4GPK08L78,...,A3QXKTFLOU1A1G,A3SWR6FYZ9ECN,A3TE5OTAQOXS65,A3TGY3ANGBNK7C,A3U41P3MHAEXYU,A4IA1Q5W73W06,A4YZH1ZTGR4J0,A5JZNVV4TEAWU,A6HLH3PIEDC9T,A6SR7U2PNGN0N,A7AO0PBCKSW82,A8A4LKHUV6QB0,A9G85AO0S9UTB,AA5HLEE8ND8TJ,AB0WXE9RJYX2T,ABDR6IJ93HFIO,ACOXR0GL8E2Y7,AD9HAE7DPTQDG,ADDT2MU773IYL,AGSVJM3JA24FV,AH32373XJKIY8,AJROLS3AD3OLL,AJRYPPA566O8I,AJVQTGMBF8PL1,AKU87CDPNQFJ,ALY4MQYVAEE4U,AM4XZBQH36O2M,ANOSVLTGRKABQ,ANZ16BHO3WGU,APKYBLUK40B6A,APTITVI3ZZ3K3,AR9RKLQQC2L6K,ASKZO80Z1RKTR,ASVY5XSYJ1XOE,AU510CVD9XDG,AW4NFCPN81CS3,AWYHVX4ZAVGV2,AX5KL9LWC1TG6,AXIO51VZN815F,AYDMB9RBJMTBR
asin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
B000JMLBHU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B001BXNQ2O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B002AJ7X2C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0030CMJEK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B0038KX8S0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ratingsd.shape

(1104, 216)

In [ ]:

from sklearn.model_selection import train_test_split
traind, testd = train_test_split(ratingsd, test_size=0.30,random_state=42)

In [ ]:
train = traind.to_numpy()
test = testd.to_numpy()

In [ ]:
sparsity = float(len(train.nonzero()[0]))
sparsity /= (train.shape[0] * train.shape[1])
sparsity *= 100
print ('Sparsity: {:5.2f}%'.format(sparsity))

Sparsity:  0.71%


In [ ]:
def item_similarity(ratings, epsilon=1e-9):
    # epsilon -> for handling dived-by-zero errors
    sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

In [ ]:
item_sim = item_similarity(train)

In [ ]:
def predict_item(ratings, similarity):
    return ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])

In [ ]:
item_prediction = predict_item(train, item_sim)

In [ ]:
item_prediction[:4, :4]

array([[1.22877004e-11, 1.86269525e-01, 1.14434989e-11, 1.11501067e-11],
       [1.44626953e-11, 1.46160215e-11, 1.34690653e-11, 1.31237409e-11],
       [2.97453732e-11, 3.00607185e-11, 2.77017778e-11, 2.69915503e-11],
       [6.07255151e-11, 6.13692962e-11, 5.65534921e-11, 2.19843926e+00]])

In [ ]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [ ]:
print ('Item-based CF MSE: ' + str(get_mse(item_prediction, test)))

Item-based CF MSE: 19.302640715714443


In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
r_mat_tr=svd.fit_transform(traind) 
print(svd.explained_variance_ratio_)  
print(svd.explained_variance_ratio_.sum())

#pm=pd.DataFrame(cosine_similarity(r_mat_tr))
#pm.head()
ctrain = cosine_similarity(r_mat_tr)

[0.02834594 0.02487422 0.01728777 0.01917014 0.01679008 0.01560585
 0.0151438  0.01503328 0.01450151 0.01375018 0.01322067 0.01264874
 0.01216288 0.01213211 0.01088275 0.01088754 0.0109368  0.01055127
 0.01055451 0.01026407 0.01019343 0.0100883  0.01000507 0.00977913
 0.00968173 0.00954465 0.00941933 0.00925724 0.00898387 0.00898501
 0.0087169  0.00835679 0.00841972 0.00835713 0.0082135  0.00817363
 0.00818019 0.00791394 0.00793514 0.00774985 0.0077055  0.00761858
 0.00762692 0.00750276 0.00725557 0.00715011 0.00712578 0.0070079
 0.00689025 0.00682509]
0.545407118772279


In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
r_mat_tr=svd.fit_transform(testd) 
print(svd.explained_variance_ratio_)  
print(svd.explained_variance_ratio_.sum())

#pmtt=pd.DataFrame(cosine_similarity(r_mat_tr))
#print (pmtt[:2])
#pmtt.head()

[0.03509543 0.03759686 0.02895198 0.02518443 0.02245581 0.02079457
 0.019341   0.01699439 0.01758556 0.01683619 0.01632832 0.01432504
 0.01510865 0.01461965 0.01452411 0.0137423  0.01371695 0.01347772
 0.01340526 0.01288502 0.01251401 0.01223949 0.01216131 0.01134466
 0.0114169  0.01109068 0.01114789 0.01069172 0.01074559 0.01046162
 0.01037604 0.00968051 0.00960401 0.0095284  0.0093674  0.00933171
 0.00912085 0.00905085 0.00895132 0.00890627 0.00888717 0.00876833
 0.00865322 0.00842112 0.00822813 0.00778818 0.00765033 0.00769073
 0.00729853 0.0070083 ]
0.6710944994670791


In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

svd = TruncatedSVD(n_components=50, n_iter=7, random_state=42)
r_mat_tr=svd.fit_transform(testd) 
print(svd.explained_variance_ratio_)  
print(svd.explained_variance_ratio_.sum())

#pmtt=pd.DataFrame(cosine_similarity(r_mat_tr))
#print (pmtt[:2])
#pmtt.head()
ctest = cosine_similarity(r_mat_tr)

[0.03509543 0.03759686 0.02895198 0.02518443 0.02245581 0.02079457
 0.019341   0.01699439 0.01758556 0.01683619 0.01632832 0.01432504
 0.01510865 0.01461965 0.01452411 0.0137423  0.01371695 0.01347772
 0.01340526 0.01288502 0.01251401 0.01223949 0.01216131 0.01134466
 0.0114169  0.01109068 0.01114789 0.01069172 0.01074559 0.01046162
 0.01037604 0.00968051 0.00960401 0.0095284  0.0093674  0.00933171
 0.00912085 0.00905085 0.00895132 0.00890627 0.00888717 0.00876833
 0.00865322 0.00842112 0.00822813 0.00778818 0.00765033 0.00769073
 0.00729853 0.0070083 ]
0.6710944994670791


In [ ]:
print (' CF MSE: ' + str(get_mse(ctrain, ctest)))

 CF MSE: 0.04488844931657428


In [ ]:
df2 = df2.sort_values(by='rating')
df2 = df2.reset_index(drop=True)
count_users = df2.groupby("reviewerID", as_index=False).count()

In [ ]:
count = df2.groupby("asin", as_index=False).mean()

In [ ]:
items_df = count[['asin']]
items_df.head()
print(len(items_df))

1104


In [ ]:
users_df = count_users[['reviewerID']]
users_df.head()
print(len(users_df))

216


In [ ]:
users_list = users_df.values
len(users_list)

216

In [ ]:
df_clean_matrix = df2.pivot(index='asin', columns='reviewerID', values='rating').fillna(0)
df_clean_matrix = df_clean_matrix.T
R = (df_clean_matrix).to_numpy()
R

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
R_demeaned

array([[-0.03894928, -0.03894928, -0.03894928, ..., -0.03894928,
        -0.03894928, -0.03894928],
       [-0.05253623, -0.05253623, -0.05253623, ..., -0.05253623,
        -0.05253623, -0.05253623],
       [-0.01268116, -0.01268116, -0.01268116, ..., -0.01268116,
        -0.01268116, -0.01268116],
       ...,
       [-0.00905797, -0.00905797, -0.00905797, ..., -0.00905797,
        -0.00905797, -0.00905797],
       [-0.00815217, -0.00815217, -0.00815217, ..., -0.00815217,
        -0.00815217, -0.00815217],
       [-0.01811594, -0.01811594, -0.01811594, ..., -0.01811594,
        -0.01811594, -0.01811594]])

In [ ]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned)

In [ ]:
sigma = np.diag(sigma)

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = df_clean_matrix.columns)
preds_df['reviewerID'] = users_df
preds_df.set_index('reviewerID', inplace=True)
preds_df.head()

asin,B000JMLBHU,B001BXNQ2O,B002AJ7X2C,B0030CMJEK,B0038KX8S0,B003FK5QAG,B003O85YEM,B003TZLM3M,B003U89RS0,B003U89S66,B003XVYJ2S,B003ZUYQE8,B0041T4FA2,B0042P5DA6,B0043GX2FW,B0043GX2HU,B0045UA6F0,B00480OPOK,B004A14RI6,B004AYDJM8,B004BA5GPY,B004C44556,B004G5ZSJ8,B004GXB2DG,B004HO6ATU,B004K6MMCY,B004LROPC2,B004MSQZUU,B004N62M26,B004OA5ZTS,B004PLNH5K,B004QT6YBK,B004RVNMBW,B004S81POK,B004SUP1XO,B004TS2B4W,B004U362DC,B004UI6UI4,B004UWPNJM,B004VF69LY,...,B00K9NWRLI,B00K9ODF5E,B00K9ZLW3K,B00KA0AT06,B00KANKYJE,B00KANRBOK,B00KCE1ISM,B00KE91OBG,B00KEV76JI,B00KEZK9F2,B00KF0URBM,B00KGOE9V6,B00KHHOUKW,B00KHHX31O,B00KJHFEOG,B00KKXTQ52,B00KLJCUIK,B00KPF2M6U,B00KQ1Y1IA,B00KQA8Y7K,B00KRQCHYO,B00KTD7SPS,B00KUCK4V8,B00KUEGX16,B00KV13TKG,B00KVN6HH6,B00KVOTDRQ,B00KVU9LRW,B00KXIIZ3I,B00KXJO6BC,B00L0I2G7Q,B00L0N6LSG,B00L4QC6ME,B00L85OBBU,B00LD6IYUS,B00LDD8NDO,B00LKJMA8A,B00LKNP4T8,B00LLI4V82,B00LLWDUDK
reviewerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A10Y597K071WTQ,0.033144,0.030145,0.033009,0.184206,0.040931,0.055435,0.043612,0.032943,0.031371,0.032971,0.032791,0.033115,0.033761,0.033083,0.030940,0.031626,0.034373,-0.022792,0.025333,0.032860,0.030645,0.033420,0.021594,0.033179,0.037747,0.032324,0.032733,0.033859,0.041372,0.032324,0.034810,0.032983,0.045917,0.166841,0.033624,0.032943,0.033658,0.032733,0.032999,0.044178,...,0.033392,0.032727,0.155588,0.030351,0.024420,-0.015631,0.030351,0.033212,0.037330,0.036137,0.024378,0.032978,0.026222,0.029289,0.029289,0.000757,0.026222,0.039948,0.026948,0.034059,0.027120,0.027599,0.041991,0.031938,0.029113,0.045910,0.028375,0.034374,0.034112,0.033797,0.034374,0.005358,0.010190,0.007403,0.021501,0.018721,-0.008262,0.025373,0.042839,0.018721
A1168ILANAUMV8,0.056135,0.057110,0.056424,0.099239,0.062042,0.070821,0.058534,0.056502,0.054885,0.057536,0.057522,0.056182,0.063124,0.056235,0.060286,0.060367,0.058652,0.017355,0.050633,0.061081,0.055790,0.064282,0.052357,0.056409,0.065464,0.059750,0.056970,0.059111,0.059060,0.059750,0.061362,0.057321,0.065576,-0.001484,0.056630,0.056502,0.061694,0.056970,0.056682,0.066247,...,0.057770,0.056982,-0.002989,0.055342,0.049412,0.005043,0.055342,0.060174,0.058411,0.056731,0.053699,0.056424,0.050635,0.050226,0.050226,0.009270,0.050635,0.050979,0.053950,0.055758,0.054003,0.048388,0.064193,0.059408,0.051389,0.067537,0.064265,0.061108,0.055990,0.056784,0.061108,0.037880,0.016091,0.022714,0.049729,0.050346,0.006660,0.053443,0.051733,0.050346
A11US7VEC93CT6,0.014057,0.014482,0.014141,-0.000095,0.014309,0.015174,0.013760,0.014167,0.013701,0.014401,0.014436,0.014071,0.015631,0.014087,0.014678,0.015177,0.014642,0.007371,0.012731,0.015140,0.013346,0.015952,0.013941,0.014126,0.015873,0.014911,0.014303,0.014730,0.014857,0.014911,0.015223,0.014368,0.014403,0.002013,0.014131,0.014167,0.015306,0.014303,0.014204,0.015958,...,0.014444,0.014299,0.001536,0.014173,0.013160,0.004887,0.014173,0.015084,0.014580,0.014264,0.013860,0.014144,0.012344,0.013339,0.013339,0.006761,0.012344,0.013465,0.013767,0.011677,0.013825,0.013294,0.015568,0.014962,0.012601,0.015814,0.016307,0.015226,0.014094,0.013681,0.015226,0.010237,0.008561,0.009736,0.012494,0.013504,0.006743,0.013706,0.013722,0.013504
A135NSTCX38DEI,0.019271,0.020817,0.019403,0.002617,0.019171,0.021337,0.019230,0.019443,0.017332,0.019534,0.019948,0.019293,0.024150,0.019318,0.023079,0.022062,0.020011,-0.009957,0.014880,0.021973,0.011763,0.024715,0.016180,0.019382,0.024855,0.019781,0.019657,0.020133,0.021280,0.019781,0.021388,0.019828,0.019232,0.010321,0.019400,0.019443,0.023159,0.019657,0.019491,0.025041,...,0.019972,0.019597,0.000858,0.017695,0.017287,0.017894,0.017695,0.020787,0.020117,0.019810,0.015818,0.019407,0.027259,0.016360,0.016360,-0.017640,0.027259,0.012200,0.026492,0.050760,0.023768,0.015300,0.023872,0.019548,0.027873,0.021983,0.021862,0.020710,0.019264,0.017195,0.020710,0.054173,-0.000008,-0.017578,0.030218,0.015231,-0.014569,0.028316,0.012814,0.015231
A13D5S4OXQUPRM,0.004950,0.005086,0.004975,0.000392,0

In [ ]:
preds_df.shape

(216, 1104)

In [ ]:

def recommend_it(predictions_df, itm_df, original_ratings_df, num_recommendations=10,ruserId='A13QTZ8CIMHHG4'):
    
    # Get and sort the user's predictions
    sorted_user_predictions = predictions_df.loc[ruserId].sort_values(ascending=False)
    
    # Get the user's data and merge in the item information.
    user_data = original_ratings_df[original_ratings_df.reviewerID == ruserId]
    user_full = (user_data.merge(itm_df, how = 'left', left_on = 'asin', right_on = 'asin').
                     sort_values(['rating'], ascending=False)
                 )

    print ('User {0} has already purchased {1} items.'.format(ruserId, user_full.shape[0]))
    print ('Recommending the highest {0} predicted  items not already purchased.'.format(num_recommendations))
    
    # Recommend the highest predicted rating items that the user hasn't bought yet.
    recommendations = (itm_df[~itm_df['asin'].isin(user_full['asin'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'asin',
               right_on = 'asin').
         rename(columns = {ruserId: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )
    topk=recommendations.merge(original_ratings_df,left_index = True, right_on = 'asin',left_on='asin').drop_duplicates(
    ['asin'])[['asin']]

    return topk

In [ ]:
recommend_it(preds_df, items_df, df2, 8)

User A13QTZ8CIMHHG4 has already purchased 7 items.
Recommending the highest 8 predicted  items not already purchased.


,asin
781,B0089H8AQ2
614,B009IAMQIW
102,B00GGRBCNU
237,B009JWLKEU
993,B00C8OZB7I
583,B00D4B53UE
994,B00G5XIXDM
152,B007UAUPT4
